In [ ]:
!pip install langchain openai

# Import libs

In [3]:
from getpass import getpass
from langchain import OpenAI, PromptTemplate
from langchain.chains import LLMChain,ConversationChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [4]:
OPENAI_API_KEY = getpass()

··········


In [7]:
from langchain.memory import ConversationBufferMemory

# Required prompt templates

In [8]:
qa_template = """Answer the questions in the context given below.\
Context: {chat_history}
Keep the answer short and concise. \
Respond "Unsure about answer" if not sure about the answer.\
Question: {input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

{chat_history}
Here is a question:
{input}"""

bash_template = """If someone asks you to perform a task, \
your job is to come up with a series of bash commands \
that will perform the task. There is no need to \
put \"#!/bin/bash\" in your answer. Make sure to \
reason step by step, using this format:
\n\nQuestion: \"copy the files in the directory named \
'target' into a new directory at the same level as \
target called 'myNewDirectory'\
\"\n\nI need to take the following actions:
\n- List all files in the directory
\n- Create a new directory
\n- Copy the files from the first directory into the second directory
\n```bash\nls\nmkdir myNewDirectory\ncp -r target/* myNewDirectory\n```\n\n
That is the format. Begin!
{chat_history}
\n\nQuestion: {input}
"""

In [9]:
prompt_infos = [
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "bash",
        "description": "Good for writing bash script",
        "prompt_template": bash_template
    },
    {
        "name": "qa",
        "description": "Good for answering generic questions",
        "prompt_template": qa_template
    }
]

In [10]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)

# Initializing destination chains with memories

* verbose=True

In [11]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(
        verbose=True,
        llm=llm,
        prompt=prompt,
        memory = ConversationBufferMemory(memory_key="chat_history")
    )
    destination_chains[name] = chain

In [12]:
default_chain = ConversationChain(
    llm=llm,
    output_key="text"
)

In [13]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

# Router Prompt

In [14]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

# Chain to route to destination chains

In [15]:
from langchain import memory
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(
    llm,
    router_prompt,
)

# Final Chain

In [16]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [17]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


qa: {'input': 'What is black body radiation?'}

> Entering new LLMChain chain...
Prompt after formatting:
Answer the questions in the context given below.Context: 
Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.Question: What is black body radiation?

> Finished chain.

> Finished chain.


'\n\nBlack body radiation is the thermal electromagnetic radiation emitted from the surface of a body that is in thermal equilibrium with its environment.'

In [18]:
chain.run("How to make a new dir")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


bash: {'input': 'mkdir [name of directory]'}

> Entering new LLMChain chain...
Prompt after formatting:
If someone asks you to perform a task, your job is to come up with a series of bash commands that will perform the task. There is no need to put "#!/bin/bash" in your answer. Make sure to reason step by step, using this format:


Question: "copy the files in the directory named 'target' into a new directory at the same level as target called 'myNewDirectory'"

I need to take the following actions:

- List all files in the directory

- Create a new directory

- Copy the files from the first directory into the second directory

```bash
ls
mkdir myNewDirectory
cp -r target/* myNewDirectory
```


That is the format. Begin!



Question: mkdir [name of directory]


> Finished chain.

> Finished chain.


'\n```bash\nmkdir [name of directory]\n```'

In [19]:
chain.run("Is it related to electromagnetic radiation")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


qa: {'input': 'Is it related to electromagnetic radiation?'}

> Entering new LLMChain chain...
Prompt after formatting:
Answer the questions in the context given below.Context: Human: What is black body radiation?
AI: 

Black body radiation is the thermal electromagnetic radiation emitted from the surface of a body that is in thermal equilibrium with its environment.
Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.Question: Is it related to electromagnetic radiation?

> Finished chain.

> Finished chain.


'\n\nYes, it is related to electromagnetic radiation.'

In [20]:
chain.run("What is it related to")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


qa: {'input': 'What is it related to?'}

> Entering new LLMChain chain...
Prompt after formatting:
Answer the questions in the context given below.Context: Human: What is black body radiation?
AI: 

Black body radiation is the thermal electromagnetic radiation emitted from the surface of a body that is in thermal equilibrium with its environment.
Human: Is it related to electromagnetic radiation?
AI: 

Yes, it is related to electromagnetic radiation.
Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.Question: What is it related to?

> Finished chain.

> Finished chain.


'\n\nIt is related to electromagnetic radiation.'

In [21]:
chain.run("What are we taking about")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


qa: {'input': 'What are we talking about?'}

> Entering new LLMChain chain...
Prompt after formatting:
Answer the questions in the context given below.Context: Human: What is black body radiation?
AI: 

Black body radiation is the thermal electromagnetic radiation emitted from the surface of a body that is in thermal equilibrium with its environment.
Human: Is it related to electromagnetic radiation?
AI: 

Yes, it is related to electromagnetic radiation.
Human: What is it related to?
AI: 

It is related to electromagnetic radiation.
Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.Question: What are we talking about?

> Finished chain.

> Finished chain.


'\n\nWe are talking about black body radiation.'